# Predicting sentiment from product reviews

The goal of this assignment is to explore logistic regression and feature engineering with existing GraphLab Create functions.

In this assignment, you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative. You will:

Use SFrames to do some feature engineering
Train a logistic regression model to predict the sentiment of product reviews.

Inspect the weights (coefficients) of a trained logistic regression model.

Make a prediction (both class and probability) of sentiment for a new product review.

Given the logistic regression weights, predictors and ground truth labels, write a function to compute the accuracy of the model.

Inspect the coefficients of the logistic regression model and interpret their meanings.
Compare multiple logistic regression models.

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [1]:
import pandas as pd
import numpy as np

In [2]:
products = pd.read_csv('./amazon_baby.csv')

In [3]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [4]:
products = products.fillna({'review':''})

In [5]:
products['review_without_punctuation'] = products['review'].str.replace('[^\w\s]','')

Now, let us see a preview of what the dataset looks like.

In [6]:
products.head()

,name,review,rating,review_without_punctuation
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,These flannel wipes are OK but in my opinion n...
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...


## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [7]:
products = products[products['rating'] != 3]

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [8]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products.head()

,name,review,rating,review_without_punctuation,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [9]:
train_indices = open('module-2-assignment-train-idx.txt', 'r')
test_indices = open('module-2-assignment-test-idx.txt', 'r')

In [10]:
for line in train_indices:
    train_list = [int(x.strip()) for x in line.split(',')]
for line in test_indices:
    test_list = [int(x.strip()) for x in line.split(',')]

In [11]:
train_data = products.iloc[train_list]
test_data = products.iloc[test_list]

# Build the word count vector for each review

We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as bag-of-word features. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction. Compute the occurrences of the words in each review and collect them into a row vector. Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix train_matrix. Using the same mapping between words and columns, convert the test data into a sparse matrix test_matrix.


In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_without_punctuation'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_without_punctuation'])

In [13]:
sentiment_model_words = vectorizer.vocabulary_.keys()

# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data.

Learn a logistic regression classifier using the training data. If you are using scikit-learn, you should create an instance of the LogisticRegression class and then call the method fit() to train the classifier. This model should use the sparse word count matrix (train_matrix) as features and the column sentiment of train_data as the target. Use the default values for other parameters. Call this model sentiment_model.

There should be over 100,000 coefficients in this sentiment_model. Recall from the lecture that positive weights w_j correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. Calculate the number of positive (>= 0, which is actually nonnegative) coefficients. 

In [14]:
from sklearn import datasets, linear_model
from sklearn.linear_model import LogisticRegression

sentiment_model = LogisticRegression()

In [15]:
sentiment_model.fit(train_matrix, train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [16]:
weights = sentiment_model.coef_
weights

array([[ -1.23841781e+00,   1.66173192e-04,   2.62689148e-02, ...,
          1.17195632e-02,   3.11809533e-03,  -6.46747356e-05]])

**Quiz Question:** How many weights are >= 0?

In [17]:
num_positive_weights = np.sum(weights >= 0)
num_negative_weights = np.sum(weights < 0)

print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights

Number of positive weights: 85924 
Number of negative weights: 35817 


## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [18]:
sample_test_data = test_data[10:13]
print sample_test_data['rating']

59    5
71    2
91    1
Name: rating, dtype: int64


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [19]:
sample_test_data.iloc[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [20]:
sample_test_data.iloc[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [21]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_without_punctuation'])
scores = sentiment_model.decision_function(sample_test_matrix)
print scores

[  5.60827718  -3.15921163 -10.41343225]


# Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [22]:
sentiment_model.predict(sample_test_matrix)

array([ 1, -1, -1])

**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

In [23]:
probability = 1/(1+np.exp(-scores))
probability

array([  9.96346018e-01,   4.07298447e-02,   3.00255379e-05])

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. 

In [24]:
test_matrix = vectorizer.transform(test_data['review_without_punctuation'])
scores_test = sentiment_model.decision_function(test_matrix)
predictions = 1/(1+np.exp(-scores_test))
test_data['predictions'] = predictions
twenty_most_positive_reviews = test_data.sort('predictions', ascending = False).iloc[0:20]
twenty_most_positive_reviews

/Users/jchumley/anaconda/envs/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jchumley/anaconda/envs/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,name,review,rating,review_without_punctuation,sentiment,predictions
97325,Freemie Hands-Free Concealable Breast Pump Col...,I absolutely love this product. I work as a C...,5,I absolutely love this product I work as a Cu...,1,1.0
133651,"Britax 2012 B-Agile Stroller, Red",[I got this stroller for my daughter prior to ...,4,I got this stroller for my daughter prior to t...,1,1.0
50315,"P'Kolino Silly Soft Seating in Tias, Green",I've purchased both the P'Kolino Little Reader...,4,Ive purchased both the PKolino Little Reader C...,1,1.0
180646,Mamas &amp; Papas 2014 Urbo2 Stroller - Black,After much research I purchased an Urbo2. It's...,4,After much research I purchased an Urbo2 Its e...,1,1.0
168697,Graco FastAction Fold Jogger Click Connect Str...,Graco's FastAction Jogging Stroller definitely...,5,Gracos FastAction Jogging Stroller definitely ...,1,1.0
100166,"Infantino Wrap and Tie Baby Carrier, Black Blu...",I bought this carrier when my daughter was abo...,5,I bought this carrier when my daughter was abo...,1,1.0
168081,Buttons Cloth Diaper Cover - One Size - 8 Colo...,"We are big Best Bottoms fans here, but I wante...",4,We are big Best Bottoms fans here but I wanted...,1,1.0
140816,"Diono RadianRXT Convertible Car Seat, Plum",I bought this seat for my tall (38in) and thin...,5,I bought this seat for my tall 38in and thin 2...,1,1.0
52631,Evenflo X Sport Plus Convenience Stroller - Ch...,After seeing this in Parent's Magazine and rea...,5,After seeing this in Parents Magazine and read...,1,1.0
119182,Roan Rocco Classic Pram Stroller 2-in-1 with B...,Great Pram Rocco!!!!!!I bought this pram from ...,5,Great Pram RoccoI bought this pram from Europe...,1,1.0


**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [25]:
twenty_most_neg_reviews = test_data.sort('predictions', ascending = True).iloc[0:20]
twenty_most_neg_reviews

/Users/jchumley/anaconda/envs/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,name,review,rating,review_without_punctuation,sentiment,predictions
16042,Fisher-Price Ocean Wonders Aquarium Bouncer,We have not had ANY luck with Fisher-Price pro...,2,We have not had ANY luck with FisherPrice prod...,-1,9.219351e-16
120209,Levana Safe N'See Digital Video Baby Monitor w...,This is the first review I have ever written o...,1,This is the first review I have ever written o...,-1,1.853912e-15
77072,Safety 1st Exchangeable Tip 3 in 1 Thermometer,I thought it sounded great to have different t...,1,I thought it sounded great to have different t...,-1,8.065162e-14
48694,Adiri BPA Free Natural Nurser Ultimate Bottle ...,I will try to write an objective review of the...,2,I will try to write an objective review of the...,-1,1.214262e-13
155287,VTech Communications Safe &amp; Sounds Full Co...,"This is my second video monitoring system, the...",1,This is my second video monitoring system the ...,-1,1.939524e-13
94560,The First Years True Choice P400 Premium Digit...,Note: we never installed batteries in these un...,1,Note we never installed batteries in these uni...,-1,4.310917e-13
53207,Safety 1st High-Def Digital Monitor,We bought this baby monitor to replace a diffe...,1,We bought this baby monitor to replace a diffe...,-1,3.454597e-11
81332,Cloth Diaper Sprayer--styles may vary,I bought this sprayer out of desperation durin...,1,I bought this sprayer out of desperation durin...,-1,3.874705e-11
113995,Motorola Digital Video Baby Monitor with Room ...,DO NOT BUY THIS BABY MONITOR!I purchased this ...,1,DO NOT BUY THIS BABY MONITORI purchased this m...,-1,9.051359e-11
10677,Philips AVENT Newborn Starter Set,"It's 3am in the morning and needless to say, t...",1,Its 3am in the morning and needless to say thi...,-1,1.093866e-10


**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

Step 1: Use the sentiment_model to compute class predictions.

Step 2: Count the number of data points when the predicted class labels match the ground truth labels.

Step 3: Divide the total number of correct predictions by the total number of data points in the dataset

Complete the function below to compute the classification accuracy:

In [26]:
predicted_sentiment = sentiment_model.predict(test_matrix)

In [27]:
test_data['predicted_sentiment'] = predicted_sentiment

/Users/jchumley/anaconda/envs/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [28]:
test_data['diff_sentiment'] = predicted_sentiment - test_data['sentiment']

/Users/jchumley/anaconda/envs/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [29]:
num_correctly_classified = len(test_data[test_data['diff_sentiment'] == 0])
num_correctly_classified 

31080

In [30]:
num_data_points = len(test_data.index)
num_data_points

33336

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

In [31]:
float(num_correctly_classified/num_data_points)

0.0

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [32]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [33]:
len(significant_words)

20

Compute a new set of word count vectors using only these words. The CountVectorizer class has a parameter that lets you limit the choice of words when building word count vectors:

In [34]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_without_punctuation'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_without_punctuation'])

Compute word count vectors for the training and test data and obtain the sparse matrices train_matrix_word_subset and test_matrix_word_subset, respectively.

Let's see what the first example of the dataset looks like:

## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

Let us inspect the weights (coefficients) of the simple_model. First, build a table to store (word, coefficient) pairs. If you are using SFrame with scikit-learn, you can combine words with coefficients by running

In [35]:
simple_model = LogisticRegression()
simple_model.fit(train_matrix_word_subset, train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [36]:
simple_model_coefficients = simple_model.coef_

In [37]:
simple_model_table = pd.DataFrame({'word':significant_words,
                                         'coefficients': simple_model_coefficients.flatten()})

Sort the data frame by the coefficient value in descending order.

In [38]:
simple_model_table.sort('coefficients', ascending = False)

/Users/jchumley/anaconda/envs/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,coefficients,word
6,1.673074,loves
5,1.509812,perfect
0,1.363690,love
2,1.192538,easy
1,0.944000,great
4,0.520186,little
7,0.503760,well
8,0.190909,able
3,0.085513,old
9,0.058855,car


Quiz Question: Consider the coefficients of simple_model. How many of the 20 coefficients (corresponding to the 20 significant_words) are positive for the simple_model?

In [39]:
simple_model_positive_words = simple_model_table[simple_model_table['coefficients'] >=0]
len(simple_model_positive_words)

10

Quiz Question: Are the positive words in the simple_model also positive words in the sentiment_model?

In [40]:
vocabulary = vectorizer.vocabulary_.keys()

In [41]:
sentiment_model_coeffs = {vocabulary[k]: v for k, v in enumerate(sentiment_model.coef_[0])}

In [42]:
word_dict = {k:v for k, v in sentiment_model_coeffs.items() if k in significant_words}

In [43]:
sentiment_model_table =  pd.DataFrame(word_dict.items(), columns=['word', 'new_coefficient'])

In [44]:
sentiment_table = pd.merge(simple_model_table, sentiment_model_table, how = 'left', on = 'word' )

In [45]:
sentiment_table[sentiment_table['coefficients']>=0]

,coefficients,word,new_coefficient
0,1.363690,love,-0.050235
1,0.944000,great,-0.002651
2,1.192538,easy,0.000027
3,0.085513,old,0.012463
4,0.520186,little,0.012667
5,1.509812,perfect,0.762225
6,1.673074,loves,0.003485
7,0.503760,well,0.104036
8,0.190909,able,0.002076
9,0.058855,car,0.122711


In [46]:
simple_model_positive_words

,coefficients,word
0,1.363690,love
1,0.944000,great
2,1.192538,easy
3,0.085513,old
4,0.520186,little
5,1.509812,perfect
6,1.673074,loves
7,0.503760,well
8,0.190909,able
9,0.058855,car


# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [47]:
predicted_sentiment_model_train = sentiment_model.predict(train_matrix)
train_data['predicted_sentiment_model_train'] = predicted_sentiment_model_train
accuracy_train_sentiment = round(float(sum(train_data['predicted_sentiment_model_train'] == train_data['sentiment']))/len(train_data.index),2)

/Users/jchumley/anaconda/envs/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [48]:
predicted_simple_model_train = simple_model.predict(train_matrix_word_subset)
train_data['predicted_simple_model_train'] = predicted_simple_model_train
accuracy_train_simple = round(float(sum(train_data['predicted_simple_model_train'] == train_data['sentiment']))/len(train_data.index),2)

/Users/jchumley/anaconda/envs/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Quiz Question: Which model (sentiment_model or simple_model) has higher accuracy on the TRAINING set?

In [49]:
accuracy_train_sentiment < accuracy_train_simple

False

In [50]:
accuracy_train_sentiment

0.97

In [51]:
accuracy_train_simple

0.87

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [52]:
predicted_sentiment_model_test = sentiment_model.predict(test_matrix)
test_data['predicted_sentiment_model_test'] = predicted_sentiment_model_test
accuracy_test_sentiment = round(float(sum(test_data['predicted_sentiment_model_test'] == test_data['sentiment']))/len(test_data.index),2)

/Users/jchumley/anaconda/envs/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [53]:
accuracy_test_sentiment

0.93

In [54]:
predicted_simple_model_test = simple_model.predict(test_matrix_word_subset)
test_data['predicted_simple_model_test'] = predicted_simple_model_test
accuracy_test_simple = round(float(sum(test_data['predicted_simple_model_test'] == test_data['sentiment']))/len(test_data.index),2)

/Users/jchumley/anaconda/envs/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [55]:
accuracy_test_simple

0.87

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

In [56]:
accuracy_test_simple < accuracy_test_sentiment

True

Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [57]:
round(float(sum(test_data['sentiment'] ==1))/len(test_data.index), 2)

0.84

**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?